In [5]:
import numpy as np

In [11]:
from dipy.io.stateful_tractogram import StatefulTractogram, Space
from tqdm import tqdm
from AFQ.utils.streamlines import bundles_to_tgram
from AFQ.api import make_bundle_dict

BUNDLE_MAT_2_PYTHON = \
    {'Right Corticospinal': 'CST_R', 'Left Corticospinal': 'CST_L',
     'Right Uncinate': 'UNC_R', 'Left Uncinate': 'UNC_L',
     'Left IFOF': 'IFO_L', 'Right IFOF': 'IFO_R',
     'Right Arcuate': 'ARC_R', 'Left Arcuate': 'ARC_L',
     'Right Thalamic Radiation': 'ATR_R', 'Left Thalamic Radiation': 'ATR_L',
     'Right Cingulum Cingulate': 'CGC_R', 'Left Cingulum Cingulate': 'CGC_L',
     'Right Cingulum Hippocampus': 'HCC_R',
     'Left Cingulum Hippocampus': 'HCC_L',
     'Callosum Forceps Major': 'FP', 'Callosum Forceps Minor': 'FA',
     'Right ILF': 'ILF_R', 'Left ILF': 'ILF_L',
     'Right SLF': 'SLF_R', 'Left SLF': 'SLF_L'}

BUNDLES = ["ATR", "CGC", "CST", "IFO", "ILF", "SLF", "ARC", "UNC",
           "FA", "FP", "HCC"]

class MatlabFileTracking():
    def __init__(self, mat_file, fg_ref):
        self.mat_file = mat_file
        self.fg_ref = fg_ref
    def __iter__(self):
        for i in tqdm(range(self.fg_ref.shape[1])):
            yield self.mat_file[self.fg_ref[0, i]][:]

def track_matlab_file(mat_file, reference):
    fiber_groups = {}
    for i in range(mat_file["fg"]["name"].shape[0]):
        name_data = mat_file[mat_file["fg"]["name"][i][0]]
        name = ''.join(chr(i) for i in name_data[:])
        if name in BUNDLE_MAT_2_PYTHON.keys():
            bundle_ref = mat_file[mat_file["fg"]["fibers"][i][0]]
            tracker = MatlabFileTracking(mat_file, bundle_ref)
            fiber_groups[BUNDLE_MAT_2_PYTHON[name]] =\
                StatefulTractogram(tracker, reference, Space.RASMM)

    return bundles_to_tgram(fiber_groups, make_bundle_dict(BUNDLES), reference)

# could be T1 weighted or diffusion, try each then open up b0 volume
# not in volume coordinates (lets compare to get_streamlines)
# origin might not matter, try one of center / corner and see if they shift (prolly by half a voxel)

In [12]:
import os.path as op

import h5py

from dipy.io.streamline import save_tractogram
import nibabel as nib

mat_file = h5py.File(op.join(
    op.expanduser('~'),
    'Downloads',
    "MoriGroups.mat"), "r")
img = nib.load(op.join(op.expanduser('~'), 'Downloads', f"dwi.nii.gz"))

sft = track_matlab_file(mat_file, img)

mat_file.close()
save_tractogram(sft, f"m2p_tracts/tractometry-test.trk", bbox_valid_check=False)

100%|██████████| 974/974 [00:00<00:00, 2477.69it/s]
INFO:dipy.data.fetcher:Dataset is already in place. If you want to fetch it again please first remove the folder /Users/john/AFQ_data/templates 
INFO:dipy.data.fetcher:Dataset is already in place. If you want to fetch it again please first remove the folder /Users/john/AFQ_data/callosum_templates 


True